In [1]:
%cd /content/drive/My Drive/Signate/JLeague_predict_audience

/content/drive/My Drive/Signate/JLeague_predict_audience


In [0]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from pandas import Series
import re

In [189]:
#訓練データ
train = pd.read_csv('./data/train.csv')
train_add = pd.read_csv('./data/train_add.csv')
train = pd.concat([train, train_add],ignore_index=True).sort_values('id')
train.index=list(range(len(train)))
train

,id,y,year,stage,match,gameday,time,home,away,stadium,tv
0,13994,18250,2012,Ｊ１,第１節第１日,03/10(土),14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ総合
1,13995,24316,2012,Ｊ１,第１節第１日,03/10(土),14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ ４）／ＮＨＫ名古屋
2,13996,17066,2012,Ｊ１,第１節第１日,03/10(土),14:04,ガンバ大阪,ヴィッセル神戸,万博記念競技場,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ １）／ＮＨＫ大阪
3,13997,29603,2012,Ｊ１,第１節第１日,03/10(土),14:06,サンフレッチェ広島,浦和レッズ,エディオンスタジアム広島,スカパー／ｅ２／スカパー光／ＮＨＫ広島
4,13998,25353,2012,Ｊ１,第１節第１日,03/10(土),14:04,コンサドーレ札幌,ジュビロ磐田,札幌ドーム,スカパー／ｅ２／スカパー光（スカイ・Ａ ｓｐｏｒｔｓ＋）／ＮＨＫ札幌
...,...,...,...,...,...,...,...,...,...,...,...
1948,16234,7842,2014,Ｊ２,第２４節第１日,07/30(水),19:04,ジュビロ磐田,カマタマーレ讃岐,ヤマハスタジアム（磐田）,スカパー！／スカパー！プレミアムサービス
1949,16235,5212,2014,Ｊ２,第２４節第１日,07/30(水),19:04,ＦＣ岐阜,ファジアーノ岡山,岐阜メモリアルセンター長良川競技場,スカパー！／スカパー！プレミアムサービス
1950,16236,3883,2014,Ｊ２,第２４節第１日,07/30(水),19:03,アビスパ福岡,湘南ベルマーレ,レベルファイブスタジアム,スカパー！／スカパー！プレミアムサービス
1951,16237,5028,2014,Ｊ２,第２４節第１日,07/30(水),19:03,大分トリニータ,栃木ＳＣ,大分銀行ドーム,スカパー！／スカパー！プレミアムサービス


In [190]:
stadium = pd.read_csv('./data/stadium.csv')
print(len(stadium))
stadium.head()

59


,name,address,capa
0,名古屋市瑞穂陸上競技場,愛知県名古屋市瑞穂区山下通5-1,20000
1,豊田スタジアム,愛知県豊田市千石町7-2,40000
2,フクダ電子アリーナ,千葉県千葉市中央区川崎町1-20,18500
3,日立柏サッカー場,千葉県柏市日立台1-2-50,15349
4,ニンジニアスタジアム,愛媛県松山市上野町乙46,15576


In [192]:
#試合時間をfloat型に
time_list = []
for time in train['time']:
  time_str = re.findall('[0-9]+',str(time))
  time_list.append(float(time_str[0]+time_str[1]))
train['time_id'] = time_list

#リーグカテゴリーをfloat型に
stage_list=[]
for stage in train['stage']:
  if stage=='Ｊ１':
    stage_list.append(float(1))
  elif stage=='Ｊ２':
    stage_list.append(float(2))
train['stage_id'] = stage_list

#試合日をfloat型に
day_list = []
for day in train['gameday']:
  day_str = re.findall('[0-9]+',str(day))
  day_list.append(float(day_str[0]+day_str[1]))
train['gameday_id'] = day_list

#capa計算
capa_list = []
for name_stadium in train['stadium']:
  capa = float(stadium[stadium['name']== name_stadium]['capa'])
  capa_list.append(capa)
train['capa'] = capa_list

#tv放映数
tv_num_list = []
for tv in train['tv']:
  tv_str = str(tv)
  tv_num_list.append(float(len(re.findall('[／]+',tv_str))+1))
train['tv_num'] = tv_num_list
train.head()

,id,y,year,stage,match,gameday,time,home,away,stadium,tv,time_id,stage_id,gameday_id,capa,tv_num
0,13994,18250,2012,Ｊ１,第１節第１日,03/10(土),14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ総合,1404.0,1.0,310.0,19694.0,4.0
1,13995,24316,2012,Ｊ１,第１節第１日,03/10(土),14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ ４）／ＮＨＫ名古屋,1404.0,1.0,310.0,40000.0,4.0
2,13996,17066,2012,Ｊ１,第１節第１日,03/10(土),14:04,ガンバ大阪,ヴィッセル神戸,万博記念競技場,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ １）／ＮＨＫ大阪,1404.0,1.0,310.0,21000.0,4.0
3,13997,29603,2012,Ｊ１,第１節第１日,03/10(土),14:06,サンフレッチェ広島,浦和レッズ,エディオンスタジアム広島,スカパー／ｅ２／スカパー光／ＮＨＫ広島,1406.0,1.0,310.0,50000.0,4.0
4,13998,25353,2012,Ｊ１,第１節第１日,03/10(土),14:04,コンサドーレ札幌,ジュビロ磐田,札幌ドーム,スカパー／ｅ２／スカパー光（スカイ・Ａ ｓｐｏｒｔｓ＋）／ＮＨＫ札幌,1404.0,1.0,310.0,39232.0,4.0


In [193]:
team_index = sorted(train['home'].value_counts().index)
team_dict = dict(zip(team_index, list(range(1,len(team_index)+1))))
team_dict

{'アビスパ福岡': 1,
 'アルビレックス新潟': 2,
 'カターレ富山': 3,
 'カマタマーレ讃岐': 4,
 'ガイナーレ鳥取': 5,
 'ガンバ大阪': 6,
 'ギラヴァンツ北九州': 7,
 'コンサドーレ札幌': 8,
 'サガン鳥栖': 9,
 'サンフレッチェ広島': 10,
 'ザスパクサツ群馬': 11,
 'ザスパ草津': 12,
 'ジェフユナイテッド千葉': 13,
 'ジュビロ磐田': 14,
 'セレッソ大阪': 15,
 'ファジアーノ岡山': 16,
 'ベガルタ仙台': 17,
 'モンテディオ山形': 18,
 'ロアッソ熊本': 19,
 'ヴァンフォーレ甲府': 20,
 'ヴィッセル神戸': 21,
 '京都サンガF.C.': 22,
 '名古屋グランパス': 23,
 '大分トリニータ': 24,
 '大宮アルディージャ': 25,
 '川崎フロンターレ': 26,
 '徳島ヴォルティス': 27,
 '愛媛ＦＣ': 28,
 '東京ヴェルディ': 29,
 '松本山雅ＦＣ': 30,
 '柏レイソル': 31,
 '栃木ＳＣ': 32,
 '横浜Ｆ・マリノス': 33,
 '横浜ＦＣ': 34,
 '水戸ホーリーホック': 35,
 '浦和レッズ': 36,
 '清水エスパルス': 37,
 '湘南ベルマーレ': 38,
 '鹿島アントラーズ': 39,
 'ＦＣ岐阜': 40,
 'ＦＣ東京': 41,
 'ＦＣ町田ゼルビア': 42,
 'Ｖ・ファーレン長崎': 43}

In [194]:
home_list = []
for home in train['home']:
  home_list.append(float(team_dict[home]))
train['home_id'] = home_list

away_list = []
for away in train['away']:
  away_list.append(float(team_dict[away]))
train['away_id'] = away_list
train.head()

,id,y,year,stage,match,gameday,time,home,away,stadium,tv,time_id,stage_id,gameday_id,capa,tv_num,home_id,away_id
0,13994,18250,2012,Ｊ１,第１節第１日,03/10(土),14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ総合,1404.0,1.0,310.0,19694.0,4.0,17.0,39.0
1,13995,24316,2012,Ｊ１,第１節第１日,03/10(土),14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ ４）／ＮＨＫ名古屋,1404.0,1.0,310.0,40000.0,4.0,23.0,37.0
2,13996,17066,2012,Ｊ１,第１節第１日,03/10(土),14:04,ガンバ大阪,ヴィッセル神戸,万博記念競技場,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ １）／ＮＨＫ大阪,1404.0,1.0,310.0,21000.0,4.0,6.0,21.0
3,13997,29603,2012,Ｊ１,第１節第１日,03/10(土),14:06,サンフレッチェ広島,浦和レッズ,エディオンスタジアム広島,スカパー／ｅ２／スカパー光／ＮＨＫ広島,1406.0,1.0,310.0,50000.0,4.0,10.0,36.0
4,13998,25353,2012,Ｊ１,第１節第１日,03/10(土),14:04,コンサドーレ札幌,ジュビロ磐田,札幌ドーム,スカパー／ｅ２／スカパー光（スカイ・Ａ ｓｐｏｒｔｓ＋）／ＮＨＫ札幌,1404.0,1.0,310.0,39232.0,4.0,8.0,14.0


In [195]:
#試合時のコンディション等について
condition = pd.read_csv('./data/condition.csv')
condition_add = pd.read_csv('./data/condition_add.csv')
condition = pd.concat([condition, condition_add]).sort_values('id')
condition.index=list(range(len(condition)))
print(len(condition))
condition.head()

2304


,id,home_score,away_score,weather,temperature,humidity,referee,home_team,home_01,home_02,home_03,home_04,home_05,home_06,home_07,home_08,home_09,home_10,home_11,away_team,away_01,away_02,away_03,away_04,away_05,away_06,away_07,away_08,away_09,away_10,away_11
0,13994,1,0,雨,3.8,66%,木村 博之,ベガルタ仙台,林 卓人,菅井 直樹,鎌田 次郎,上本 大海,田村 直也,富田 晋伍,角田 誠,太田 吉彰,関口 訓充,ウイルソン,赤嶺 真吾,鹿島アントラーズ,曽ヶ端 準,新井場 徹,岩政 大樹,中田 浩二,アレックス,青木 剛,増田 誓志,小笠原 満男,本山 雅志,大迫 勇也,ジュニーニョ
1,13995,1,0,屋内,12.4,43%,西村 雄一,名古屋グランパス,楢﨑 正剛,田中 隼磨,田中 マルクス闘莉王,増川 隆洋,阿部 翔平,中村 直志,ダニルソン,藤本 淳吾,金崎 夢生,ケネディ,玉田 圭司,清水エスパルス,林 彰洋,吉田 豊,岩下 敬輔,カルフィン ヨン ア ピン,李 記帝,村松 大輔,河井 陽介,枝村 匠馬,高木 俊幸,アレックス,大前 元紀
2,13996,2,3,晴一時雨,11.3,41%,高山 啓義,ガンバ大阪,藤ヶ谷 陽介,加地 亮,中澤 聡太,今野 泰幸,藤春 廣輝,明神 智和,遠藤 保仁,佐々木 勇人,二川 孝広,ラフィーニャ,パウリーニョ,ヴィッセル神戸,徳重 健太,近藤 岳登,北本 久仁衛,伊野波 雅彦,相馬 崇人,三原 雅俊,田中 英雄,野沢 拓也,橋本 英郎,森岡 亮太,大久保 嘉人
3,13997,1,0,曇一時雨のち晴,11.4,52%,松尾 一,サンフレッチェ広島,西川 周作,森脇 良太,千葉 和彦,水本 裕貴,ミキッチ,青山 敏弘,森﨑 和幸,山岸 智,石原 直樹,髙萩 洋次郎,佐藤 寿人,浦和レッズ,加藤 順大,濱田 水輝,阿部 勇樹,槙野 智章,平川 忠亮,鈴木 啓太,山田 直輝,梅崎 司,柏木 陽介,原口 元気,田中 達也
4,13998,0,0,屋内,22.5,32%,廣瀬 格,コンサドーレ札幌,李 昊乗,高木 純平,ジェイド ノース,奈良 竜樹,岩沼 俊介,河合 竜二,山本 真希,近藤 祐介,内村 圭宏,岡本 賢明,前田 俊介,ジュビロ磐田,川口 能活,駒野 友一,チョ ビョングク,藤田 義明,山本 脩斗,小林 裕紀,山本 康裕,山田 大記,松浦 拓弥,菅沼 実,前田 遼一


In [196]:
#湿度をfloat型に
hum_list = []
for hum in condition['humidity']:
  hum_list.append(float(re.findall('[0-9]+',str(hum))[0]))
condition['humidity_id'] = hum_list
condition.head()

,id,home_score,away_score,weather,temperature,humidity,referee,home_team,home_01,home_02,home_03,home_04,home_05,home_06,home_07,home_08,home_09,home_10,home_11,away_team,away_01,away_02,away_03,away_04,away_05,away_06,away_07,away_08,away_09,away_10,away_11,humidity_id
0,13994,1,0,雨,3.8,66%,木村 博之,ベガルタ仙台,林 卓人,菅井 直樹,鎌田 次郎,上本 大海,田村 直也,富田 晋伍,角田 誠,太田 吉彰,関口 訓充,ウイルソン,赤嶺 真吾,鹿島アントラーズ,曽ヶ端 準,新井場 徹,岩政 大樹,中田 浩二,アレックス,青木 剛,増田 誓志,小笠原 満男,本山 雅志,大迫 勇也,ジュニーニョ,66.0
1,13995,1,0,屋内,12.4,43%,西村 雄一,名古屋グランパス,楢﨑 正剛,田中 隼磨,田中 マルクス闘莉王,増川 隆洋,阿部 翔平,中村 直志,ダニルソン,藤本 淳吾,金崎 夢生,ケネディ,玉田 圭司,清水エスパルス,林 彰洋,吉田 豊,岩下 敬輔,カルフィン ヨン ア ピン,李 記帝,村松 大輔,河井 陽介,枝村 匠馬,高木 俊幸,アレックス,大前 元紀,43.0
2,13996,2,3,晴一時雨,11.3,41%,高山 啓義,ガンバ大阪,藤ヶ谷 陽介,加地 亮,中澤 聡太,今野 泰幸,藤春 廣輝,明神 智和,遠藤 保仁,佐々木 勇人,二川 孝広,ラフィーニャ,パウリーニョ,ヴィッセル神戸,徳重 健太,近藤 岳登,北本 久仁衛,伊野波 雅彦,相馬 崇人,三原 雅俊,田中 英雄,野沢 拓也,橋本 英郎,森岡 亮太,大久保 嘉人,41.0
3,13997,1,0,曇一時雨のち晴,11.4,52%,松尾 一,サンフレッチェ広島,西川 周作,森脇 良太,千葉 和彦,水本 裕貴,ミキッチ,青山 敏弘,森﨑 和幸,山岸 智,石原 直樹,髙萩 洋次郎,佐藤 寿人,浦和レッズ,加藤 順大,濱田 水輝,阿部 勇樹,槙野 智章,平川 忠亮,鈴木 啓太,山田 直輝,梅崎 司,柏木 陽介,原口 元気,田中 達也,52.0
4,13998,0,0,屋内,22.5,32%,廣瀬 格,コンサドーレ札幌,李 昊乗,高木 純平,ジェイド ノース,奈良 竜樹,岩沼 俊介,河合 竜二,山本 真希,近藤 祐介,内村 圭宏,岡本 賢明,前田 俊介,ジュビロ磐田,川口 能活,駒野 友一,チョ ビョングク,藤田 義明,山本 脩斗,小林 裕紀,山本 康裕,山田 大記,松浦 拓弥,菅沼 実,前田 遼一,32.0


In [197]:
weather_index = list(condition['weather'].value_counts().index)
weather_dict = dict(zip(sorted(weather_index), list(range(1,len(weather_index)+1))))
weather_dict

{'屋内': 1,
 '晴': 2,
 '晴のち曇': 3,
 '晴のち曇一時雨': 4,
 '晴のち曇時々雨': 5,
 '晴のち雨': 6,
 '晴のち雪': 7,
 '晴一時曇': 8,
 '晴一時雨': 9,
 '晴時々曇': 10,
 '晴時々雨': 11,
 '晴時々雪': 12,
 '曇': 13,
 '曇のち晴': 14,
 '曇のち雨': 15,
 '曇のち雪': 16,
 '曇のち雷雨': 17,
 '曇一時晴': 18,
 '曇一時晴一時雨': 19,
 '曇一時雨': 20,
 '曇一時雨のち晴': 21,
 '曇一時雷雨のち曇': 22,
 '曇時々晴': 23,
 '曇時々晴一時雨': 24,
 '曇時々雨': 25,
 '曇時々雨のち晴': 26,
 '雨': 27,
 '雨のち晴': 28,
 '雨のち曇': 29,
 '雨のち曇一時雨': 30,
 '雨のち曇時々晴': 31,
 '雨時々晴': 32,
 '雨時々曇': 33,
 '雪': 34,
 '雪のち雨': 35}

In [198]:
weather_list = []
for weather_str in condition['weather']:
  weather_list.append(float(weather_dict[weather_str]))
condition['weather_id']=weather_list
condition.head()

,id,home_score,away_score,weather,temperature,humidity,referee,home_team,home_01,home_02,home_03,home_04,home_05,home_06,home_07,home_08,home_09,home_10,home_11,away_team,away_01,away_02,away_03,away_04,away_05,away_06,away_07,away_08,away_09,away_10,away_11,humidity_id,weather_id
0,13994,1,0,雨,3.8,66%,木村 博之,ベガルタ仙台,林 卓人,菅井 直樹,鎌田 次郎,上本 大海,田村 直也,富田 晋伍,角田 誠,太田 吉彰,関口 訓充,ウイルソン,赤嶺 真吾,鹿島アントラーズ,曽ヶ端 準,新井場 徹,岩政 大樹,中田 浩二,アレックス,青木 剛,増田 誓志,小笠原 満男,本山 雅志,大迫 勇也,ジュニーニョ,66.0,27.0
1,13995,1,0,屋内,12.4,43%,西村 雄一,名古屋グランパス,楢﨑 正剛,田中 隼磨,田中 マルクス闘莉王,増川 隆洋,阿部 翔平,中村 直志,ダニルソン,藤本 淳吾,金崎 夢生,ケネディ,玉田 圭司,清水エスパルス,林 彰洋,吉田 豊,岩下 敬輔,カルフィン ヨン ア ピン,李 記帝,村松 大輔,河井 陽介,枝村 匠馬,高木 俊幸,アレックス,大前 元紀,43.0,1.0
2,13996,2,3,晴一時雨,11.3,41%,高山 啓義,ガンバ大阪,藤ヶ谷 陽介,加地 亮,中澤 聡太,今野 泰幸,藤春 廣輝,明神 智和,遠藤 保仁,佐々木 勇人,二川 孝広,ラフィーニャ,パウリーニョ,ヴィッセル神戸,徳重 健太,近藤 岳登,北本 久仁衛,伊野波 雅彦,相馬 崇人,三原 雅俊,田中 英雄,野沢 拓也,橋本 英郎,森岡 亮太,大久保 嘉人,41.0,9.0
3,13997,1,0,曇一時雨のち晴,11.4,52%,松尾 一,サンフレッチェ広島,西川 周作,森脇 良太,千葉 和彦,水本 裕貴,ミキッチ,青山 敏弘,森﨑 和幸,山岸 智,石原 直樹,髙萩 洋次郎,佐藤 寿人,浦和レッズ,加藤 順大,濱田 水輝,阿部 勇樹,槙野 智章,平川 忠亮,鈴木 啓太,山田 直輝,梅崎 司,柏木 陽介,原口 元気,田中 達也,52.0,21.0
4,13998,0,0,屋内,22.5,32%,廣瀬 格,コンサドーレ札幌,李 昊乗,高木 純平,ジェイド ノース,奈良 竜樹,岩沼 俊介,河合 竜二,山本 真希,近藤 祐介,内村 圭宏,岡本 賢明,前田 俊介,ジュビロ磐田,川口 能活,駒野 友一,チョ ビョングク,藤田 義明,山本 脩斗,小林 裕紀,山本 康裕,山田 大記,松浦 拓弥,菅沼 実,前田 遼一,32.0,1.0


In [0]:
#予測データ
test = pd.read_csv('./data/test.csv')
test

,id,year,stage,match,gameday,time,home,away,stadium,tv
0,15822,2014,Ｊ１,第１８節第１日,08/02(土),19:04,ベガルタ仙台,大宮アルディージャ,ユアテックスタジアム仙台,スカパー！／スカパー！プレミアムサービス
1,15823,2014,Ｊ１,第１８節第１日,08/02(土),18:34,鹿島アントラーズ,サンフレッチェ広島,県立カシマサッカースタジアム,スカパー！／スカパー！プレミアムサービス
2,15824,2014,Ｊ１,第１８節第１日,08/02(土),19:04,浦和レッズ,ヴィッセル神戸,埼玉スタジアム２００２,スカパー！／スカパー！プレミアムサービス／ＮＨＫ ＢＳ１／テレ玉
3,15825,2014,Ｊ１,第１８節第１日,08/02(土),19:03,柏レイソル,川崎フロンターレ,日立柏サッカー場,スカパー！／スカパー！プレミアムサービス
4,15827,2014,Ｊ１,第１８節第１日,08/02(土),19:03,アルビレックス新潟,セレッソ大阪,デンカビッグスワンスタジアム,スカパー！／スカパー！プレミアムサービス
...,...,...,...,...,...,...,...,...,...,...
308,16432,2014,Ｊ２,第４２節第１日,11/23(日・祝),14:04,カターレ富山,ファジアーノ岡山,富山県総合運動公園陸上競技場,スカパー！／スカパー！プレミアムサービス
309,16433,2014,Ｊ２,第４２節第１日,11/23(日・祝),14:04,京都サンガF.C.,ＦＣ岐阜,京都市西京極総合運動公園陸上競技場兼球技場,スカパー！／スカパー！プレミアムサービス／ＮＨＫ京都／ＮＨＫ岐阜／ＫＢＳ京都（録）
310,16434,2014,Ｊ２,第４２節第１日,11/23(日・祝),14:03,カマタマーレ讃岐,ジェフユナイテッド千葉,香川県立丸亀競技場,スカパー！／スカパー！プレミアムサービス
311,16435,2014,Ｊ２,第４２節第１日,11/23(日・祝),14:04,アビスパ福岡,ロアッソ熊本,レベルファイブスタジアム,スカパー！／スカパー！プレミアムサービス


In [15]:
add_2014 = pd.read_csv('./data/2014_add.csv')
print(len(add_2014))
add_2014.head()

38


,id,year,stage,match,gameday,time,home,away,stadium,tv
0,15826,2014,Ｊ１,第１８節第１日,08/02(土),18:34,ＦＣ東京,清水エスパルス,味の素スタジアム,スカパー！／スカパー！プレミアムサービス／ＴＯＫＹＯ ＭＸ
1,15835,2014,Ｊ１,第１９節第１日,08/09(土),18:04,清水エスパルス,徳島ヴォルティス,ＩＡＩスタジアム日本平,スカパー！／スカパー！プレミアムサービス
2,15837,2014,Ｊ１,第１９節第１日,08/09(土),19:04,セレッソ大阪,ＦＣ東京,ヤンマースタジアム長居,スカパー！／スカパー！プレミアムサービス
3,15839,2014,Ｊ１,第１９節第２日,08/11(月),19:04,サンフレッチェ広島,サガン鳥栖,エディオンスタジアム広島,スカパー！／スカパー！プレミアムサービス
4,15872,2014,Ｊ１,第２３節第１日,09/13(土),19:04,アルビレックス新潟,ベガルタ仙台,デンカビッグスワンスタジアム,スカパー！／スカパー！プレミアムサービス


In [0]:
sample = pd.read_csv('./data/sample_submit.csv')
sample

,15822,10662.3951191
0,15823,10662.395119
1,15824,10662.395119
2,15825,10662.395119
3,15827,10662.395119
4,15828,10662.395119
...,...,...
307,16432,10662.395119
308,16433,10662.395119
309,16434,10662.395119
310,16435,10662.395119
